In [1]:
import scipy.stats as stats
import random
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set_context("notebook", font_scale=1.5)
%matplotlib inline

/Users/Jake/anaconda/envs/snakes/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [ ]:
def leapfrog_step(U,grad_U,q,q_list,p,p_list,stepsize,num_steps):
    
    # Half step for momentum at the beginning
    p = p - stepsize*grad_U(q,Sigma)/2
    
    # Alternate full steps for position and momentum
    for i in range(num_steps):
        
        # Make a full step for the position
        q = q + stepsize*p
        q_list = np.vstack((q_list,q))
        
        # Make a full step for the momentum, except at end of trajectory
        if (i!=steps-1):
            p = p - stepsize*grad_U(q,Sigma)
            p_list = np.vstack((p_list,p))
    
    # Make a half step for momentum at the end
    p = p - stepsize*grad_U(q,Sigma)/2
    p_list = np.vstack((p_list,p))
    
    return(q,q_list,p,p_list)

In [ ]:
def leapfrog_window(U, grad_U, current_q, Sigma, stepsize=0.25, steps=25, W = 10):
    '''
    Performs the leapfrog algorithm for HMC, with 1 accept/reject step
    '''
    
    q = current_q
    p = np.random.normal(0,1,q.size) # independent statndard normal variates
    current_p = p
    q_list=q
    p_list=p
    
    # Half step for momentum at the beginning
    p = p - stepsize*grad_U(q,Sigma)/2
    
    # Alternate full steps for position and momentum
    for i in range(num_steps):
        
        # Make a full step for the position
        q = q + stepsize*p
        q_list = np.vstack((q_list,q))
        
        # Make a full step for the momentum, except at end of trajectory
        if (i!=steps-1):
            p = p - stepsize*grad_U(q,Sigma)
            p_list = np.vstack((p_list,p))
    
    # Make a half step for momentum at the end
    p = p - stepsize*grad_U(q,Sigma)/2
    p_list = np.vstack((p_list,p))   
    
    
    reject_window = np.zeros(W,Sigma.shape[0])

    #Get the rejection window
    for i in range(W):
        
    
    
    # Negate momentum at end of trajectory to make the proposal symmetric
    p = -p
    
    # Evaluate potential and kinetic energies at start and end of trajectory
    current_U = U(current_q,Sigma)
    current_K = sum(current_p**2)/2
    proposed_U = U(q,Sigma)
    proposed_K = sum(p**2)/2

    # Accept or reject the state at end of trajectory, returning either the postion at the end
    # of the trajectory or the initial position
    if (np.random.uniform(0,1) < np.exp(current_U-proposed_U+current_K-proposed_K)):
        return(q,p_list,q_list) # Accept
    else:
        return(current_q,p_list,q_list) # Reject
    